In [33]:
!pip3 install pyspark
import pyspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
spark = pyspark.sql.SparkSession.builder\
        .master('local[*]')\
        .appName('vikram_Midterm_Q4')\
        .getOrCreate()

In [35]:
sc = spark.sparkContext
sc.setLogLevel("ERROR")
sc

<SparkContext master=local[*] appName=vikram_Midterm_Q4>

In [36]:
#Top 5 items bought between the time period of 9:00AM inclusive and 11:00PM exclusive
from pyspark.sql.functions import *
datata=spark.read.option("header",True).format("csv").load("BreadBasket_DMS.csv")
data.createOrReplaceTempView("datatable")
data1 = spark.sql("Select Item, hour(Time), count(Item) as QtySold from datatable where hour(time)>=9 and hour(time) < 23 group by item,hour(Time) order by hour(Time), QtySold desc")

In [37]:
#TOP 2 items every Hour 
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql import functions as F

temp = Window.partitionBy("hour(Time)")
temp = temp.orderBy(desc("QtySold"))
final = data1.withColumn("rank",rank().over(temp)).withColumn("rank2",F.row_number().over(temp)).where(col("rank") <= 2)

In [38]:
final = final.select(col("item"),"hour(time)","QtySold").where(col("rank2") <= 2)
final = final.show(60)

+--------------------+----------+-------+
|                item|hour(time)|QtySold|
+--------------------+----------+-------+
|              Coffee|         9|    583|
|               Bread|         9|    400|
|              Coffee|        10|    820|
|               Bread|        10|    508|
|              Coffee|        11|    946|
|               Bread|        11|    528|
|              Coffee|        12|    740|
|               Bread|        12|    474|
|              Coffee|        13|    607|
|               Bread|        13|    340|
|              Coffee|        14|    636|
|               Bread|        14|    341|
|              Coffee|        15|    519|
|               Bread|        15|    310|
|              Coffee|        16|    321|
|               Bread|        16|    196|
|              Coffee|        17|     69|
|               Bread|        17|     46|
|Afternoon with th...|        18|     14|
|              Coffee|        18|     11|
|              Tshirt|        19| 

In [39]:
##Bottom two items every hour
from pyspark.sql.functions import row_number
from pyspark.sql import functions as F, Window

temp = Window.partitionBy("hour(Time)")
temp = temp.orderBy("QtySold")
data2=data1.withColumn("rank1",rank().over(temp)).withColumn("rank2",F.row_number().over(temp)).where(col("rank1")<=2)

In [40]:
final_end = data2.select(col("item"),"hour(time)","QtySold").where(col("rank2") <= 2)
final_end = final_end.show(60)

+--------------------+----------+-------+
|                item|hour(time)|QtySold|
+--------------------+----------+-------+
|            Art Tray|         9|      1|
|    My-5 Fruit Shoot|         9|      1|
|         Vegan Feast|        10|      1|
|    Valentine's card|        10|      1|
|        Gift voucher|        11|      1|
|Afternoon with th...|        11|      1|
|   Lemon and coconut|        12|      1|
|        Kids biscuit|        12|      1|
|              Crepes|        13|      1|
|       Bread Pudding|        13|      1|
|    Christmas common|        14|      1|
|Raspberry shortbr...|        14|      1|
|   Hearty & Seasonal|        15|      1|
|Cherry me Dried f...|        15|      1|
|   Lemon and coconut|        16|      1|
|Afternoon with th...|        16|      1|
|    Coffee granules |        17|      1|
|Afternoon with th...|        17|      1|
|            Focaccia|        18|      1|
|           Empanadas|        18|      1|
|               Juice|        19| 